In [4]:
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.models import Model
from keras.optimizers import SGD

In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
df=pd.read_csv(r'data/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv')
df['image'] = df['image']+'.jpg'
datagen=ImageDataGenerator(rescale=1./255,validation_split=0.05,
                           featurewise_center=True,featurewise_std_normalization=True,
                           shear_range=0.2,zoom_range=0.2,
                           rotation_range=10,width_shift_range=0.1,
                           height_shift_range=0.1)
IMG_HEIGHT,IMG_WIDTH = 224,224
train_generator=datagen.flow_from_dataframe(dataframe=df, directory="data/ISIC2018_Task3_Training_Input", 
                                            x_col="image", y_col=['MEL','NV','BCC','AKIEC','BKL','DF','VASC'], 
                                            class_mode="raw", 
                                            target_size=(IMG_HEIGHT,IMG_WIDTH), batch_size=100,
                                           subset='training')
valid_generator=datagen.flow_from_dataframe(dataframe=df, directory="data/ISIC2018_Task3_Training_Input", 
                                            x_col="image", y_col=['MEL','NV','BCC','AKIEC','BKL','DF','VASC'], 
                                            class_mode="raw", 
                                            target_size=(IMG_HEIGHT,IMG_WIDTH), batch_size=100,
                                           subset='validation')

Using TensorFlow backend.


Found 9515 validated image filenames.
Found 500 validated image filenames.


In [6]:
classes = ['MEL','NV','BCC','AKIEC','BKL','DF','VASC']

df1=df.drop(['image'],axis=1)
print(df1.groupby(classes).size())
grous = list(df1.groupby(classes).size())
g1=df1.groupby(classes).count()
print(grous)
print(g1)
total_weights = sum(grous)
weights1 = [x/total_weights for x in reversed(grous)]
weights = {x : weights1[x] for x in range(len(weights1))} 
print(weights)

MEL  NV   BCC  AKIEC  BKL  DF   VASC
0.0  0.0  0.0  0.0    0.0  0.0  1.0      142
                           1.0  0.0      115
                      1.0  0.0  0.0     1099
               1.0    0.0  0.0  0.0      327
          1.0  0.0    0.0  0.0  0.0      514
     1.0  0.0  0.0    0.0  0.0  0.0     6705
1.0  0.0  0.0  0.0    0.0  0.0  0.0     1113
dtype: int64
[142, 115, 1099, 327, 514, 6705, 1113]
Empty DataFrame
Columns: []
Index: [(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0), (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0), (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0), (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0), (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)]
{0: 0.11113330004992511, 1: 0.6694957563654518, 2: 0.05132301547678482, 3: 0.032651023464802795, 4: 0.10973539690464304, 5: 0.011482775836245632, 6: 0.014178731902146779}


In [ ]:
# from keras.models import Sequential
# from keras.layers import Conv2D,Activation,MaxPooling2D,Dropout,Flatten,Dense
# from keras import optimizers

# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(7, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# # model = Sequential()
# # model.add(Conv2D(32, (3, 3), padding='same',
# #                  input_shape=(32,32,3)))
# # model.add(Activation('relu'))
# # model.add(Conv2D(32, (3, 3)))
# # model.add(Activation('relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# # model.add(Conv2D(64, (3, 3), padding='same'))
# # model.add(Activation('relu'))
# # model.add(Conv2D(64, (3, 3)))
# # model.add(Activation('relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# # model.add(Flatten())
# # model.add(Dense(512))
# # model.add(Activation('relu'))
# # model.add(Dropout(0.5))
# # model.add(Dense(10, activation='softmax'))

# # model.compile(optimizers.rmsprop(lr=0.0001),
# # loss="categorical_crossentropy", metrics=["accuracy"])
# STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
# STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# model.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=valid_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=5,
#                    class_weight=weights)

In [30]:
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.models import Model
from keras.optimizers import SGD

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False 

#     model.add(Flatten())
#     model.add(Dense(4096, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(4096, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(1000, activation='softmax'))
# note we exclude the above final dense layers, and add the dense layers below, so we could retrain it ourselves

x = base_model.output
x = Flatten()(x) # flatten from convolution tensor output 
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', metrics=['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,
                   class_weight=weights)

Epoch 1/100


/home/user1/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/user1/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 2/95 [..............................] - ETA: 2:45 - loss: 0.6796 - accuracy: 0.3400

KeyboardInterrupt: 

In [9]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (224, 224, 3)
num_classes = 7
from keras.layers import MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile the model
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 64)       

/home/user1/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/user1/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


95/95 [==============================] - 155s 2s/step - loss: 1.8026 - accuracy: 0.6493 - val_loss: 1.1508 - val_accuracy: 0.6640
Epoch 2/100
95/95 [==============================] - 153s 2s/step - loss: 1.0436 - accuracy: 0.6694 - val_loss: 1.0549 - val_accuracy: 0.6640
Epoch 3/100
95/95 [==============================] - 154s 2s/step - loss: 0.9854 - accuracy: 0.6694 - val_loss: 0.9315 - val_accuracy: 0.6660
Epoch 4/100
95/95 [==============================] - 154s 2s/step - loss: 0.9503 - accuracy: 0.6638 - val_loss: 0.8486 - val_accuracy: 0.6640
Epoch 5/100
95/95 [==============================] - 153s 2s/step - loss: 0.9226 - accuracy: 0.6691 - val_loss: 0.8182 - val_accuracy: 0.6760
Epoch 6/100
95/95 [==============================] - 154s 2s/step - loss: 0.9097 - accuracy: 0.6700 - val_loss: 0.8376 - val_accuracy: 0.6720
Epoch 7/100
95/95 [==============================] - 154s 2s/step - loss: 0.8898 - accuracy: 0.6731 - val_loss: 0.7164 - val_accuracy: 0.6740
Epoch 8/100
95/95 

In [19]:
model.save('sam_model.h5')

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predict_this=[]
actual=[]
x,y = next(valid_generator)
for i in range(len(x)):
    #plt.imshow(x[i])
    #plt.show()
#     print('label:',y[i])
    actual.append(y[i])
    predict_this.append(x[i])
    #print('----------------------------------------------------------------------------------------')
np.set_printoptions(suppress=True)    
pred = model.predict(np.array(predict_this))

#pred = tf.cast(pred, tf.float32)
# print(pred)
# print(type(pred))
pred = np.argmax(pred, axis=1)
actual = np.argmax(actual, axis=1)
# print(pred)
# print(actual)
# if actual.all()==pred.all():
#     print('WIN!!!!!!!!!!!!!!!')
cr = classification_report(actual, pred)
cm = confusion_matrix(actual,pred)
print(cr)
print(cm)

              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.88      0.97      0.92        62
           2       0.60      0.75      0.67         8
           3       1.00      0.50      0.67         6
           4       0.69      0.60      0.64        15
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00         1

    accuracy                           0.83       100
   macro avg       0.71      0.63      0.65       100
weighted avg       0.82      0.83      0.82       100

[[ 4  1  0  0  2  0  0]
 [ 0 60  1  0  1  0  0]
 [ 0  1  6  0  1  0  0]
 [ 0  1  2  3  0  0  0]
 [ 1  5  0  0  9  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  1]]


In [50]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predict_this=[]
actual=[]
x,y = next(valid_generator)
for i in range(len(x)):
    #plt.imshow(x[i])
    #plt.show()
#     print('label:',y[i])
    actual.append(y[i])
    predict_this.append(x[i])
    #print('----------------------------------------------------------------------------------------')
np.set_printoptions(suppress=True)    
pred = model.predict(np.array(predict_this))

#pred = tf.cast(pred, tf.float32)
# print(pred)
# print(type(pred))
pred = np.argmax(pred, axis=1)
actual = np.argmax(actual, axis=1)
# print(pred)
# print(actual)
# if actual.all()==pred.all():
#     print('WIN!!!!!!!!!!!!!!!')
cr = classification_report(actual, pred)
cm = confusion_matrix(actual,pred)
print(cr)
print(cm)

              precision    recall  f1-score   support

           0       0.29      0.29      0.29         7
           1       0.85      0.94      0.89        67
           2       0.62      0.71      0.67         7
           3       0.00      0.00      0.00         5
           4       0.78      0.64      0.70        11
           5       0.00      0.00      0.00         2
           6       0.50      1.00      0.67         1

    accuracy                           0.78       100
   macro avg       0.43      0.51      0.46       100
weighted avg       0.72      0.78      0.75       100

[[ 2  4  0  0  0  0  1]
 [ 3 63  0  0  1  0  0]
 [ 1  0  5  0  1  0  0]
 [ 1  2  2  0  0  0  0]
 [ 0  4  0  0  7  0  0]
 [ 0  1  1  0  0  0  0]
 [ 0  0  0  0  0  0  1]]


In [53]:
print('loss, accuracy',model.evaluate(valid_generator))

5/5 [==============================] - 9s 2s/step
loss, accuracy [0.7176942229270935, 0.7839999794960022]
